# Upload / Download Images

In [1]:
import boto3
import botocore
from os import listdir

PROFILE = "default"
BUCKET = "dd-imagenes-prueba-rekognition"
FOLDER = "detection/" # "detection/" "identification/"
REGION = "eu-west-1"
UPLOAD = 1 

In [2]:
session = boto3.Session(profile_name=PROFILE)
s3 = session.client("s3")

In [3]:
buckets_list = s3.list_buckets()['Buckets']
print(buckets_list)
buckets_names = [buckets_list[i]['Name'] for i in range(0,len(buckets_list))]
print(bucket_names)

# Create S3 bucket called BUCKET
if BUCKET not in buckets_names:
    s3.create_bucket(Bucket=BUCKET, CreateBucketConfiguration={'LocationConstraint': REGION})

In [4]:
if UPLOAD:
    # Upload files to S3_FOLDER in BUCKET
    files = listdir(FOLDER, )
    files_names = [file for file in files if file.endswith('.jpg')]
    for name in files_names:
        s3.upload_file(FOLDER + name, BUCKET, FOLDER + name)      
else:
    # Download files to FOLDER from S3_FOLDER in BUCKET
    objects_list = s3.list_objects(Bucket = BUCKET)
    objects_keys = [key['Key'].encode("UTF-8") for key in objects_list['Contents']]
    for image in objects_keys:
        try:
            s3.download_file(BUCKET, image, FOLDER + image)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist")
            else:
                raise